In [5]:
import scipy.constants as const
eps_from_paper = 36.0
eps_in_joules = 36 * const.Boltzmann
eps_in_kcal_per_mole = eps_in_joules * const.Avogadro / 4184
print(eps_in_kcal_per_mole)

0.07153935331106996


In [ ]:
import scipy.constants as const




In [7]:
sigma_i = 3.31
sigma_j = 3.05

sigma_ij = (sigma_i + sigma_j) / 2
print(sigma_ij)

3.1799999999999997


In [5]:
# atom types & charges 
atom_info = {
    "C": (1, 0.70),
    "O": (2, -0.35),
    "X": (3, 0.964),
    "N": (4, -0.482),
}

xyz_file = "temp.xyz"

# Read XYZ and generate Atoms section
atoms = []
X_count = 0
C_count = 0
with open(xyz_file) as f:
    lines = f.readlines()
    # num_atoms = int(lines[0].strip())
    coord_lines = lines[2:]  # skip first 2 lines

    atom_id = 1
    molecule_id = 1 
    for i,line in enumerate(coord_lines):
        parts = line.split()
        element = parts[0]
        x, y, z = map(float, parts[1:4])

        if element not in atom_info:
            raise ValueError(f"Element {element} notin atom_info")

        if element == "X":
            X_count += 1
        if element == "C":
            C_count += 1

        atom_type, charge = atom_info[element]
        atoms.append((atom_id, molecule_id, atom_type, charge, x, y, z, 0, 0, 0))
        atom_id += 1
        # every three lines increase molecule id
        if (i + 1) % 3 == 0:
            molecule_id += 1


# generate Bonds section

bonds = []
bond_id = 1

for mol_start in range(0, len(atoms), 3):
    # atoms in this molecule
    a1, a2, a3 = atoms[mol_start:mol_start+3]
    # create bonds: a1-a2, a1-a3
    bonds.append((bond_id, 1, a1[0], a2[0]))
    bond_id += 1
    bonds.append((bond_id, 2, a1[0], a3[0]))
    bond_id += 1

total_atoms = len(atoms)
total_bonds = len(bonds)
print(total_atoms)
print(total_bonds)
total_atoms = X_count * 3 + C_count * 3
total_bonds = X_count * 2 + C_count * 2
total_angles = X_count + C_count
print(total_atoms)
print(total_bonds)
atoms_str = "Atoms # full\n\n" + "\n".join(" ".join(map(str, a)) for a in atoms)

data_contents = f"""\
LAMMPS data file for n2/co2

{total_atoms} atoms
4 atom types
{total_bonds} bonds
2 bond types
{total_angles} angles
2 angle types

0 90 xlo xhi
0 90 ylo yhi
0 30 zlo zhi

Masses

1 12.0107
2 15.9999
3 28.0134
4 2

Pair Coeffs # lj/cut/coul/long

1 0.0536548 2.8
2 0.15699 3.05
3 0 0
4 0.0715394 3.31

Bond Coeffs # harmonic

1 5000 1.16
2 5000 0.55

Angle Coeffs # harmonic

1 500 180
2 500 180

{atoms_str}
"""

with open("initialMD.data", "w") as f:
    f.write(data_contents)

1746
1164
1746
1164
